# Implement a basic RAG by using LCEL

In [71]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [72]:
import os
from pathlib import Path
from dotenv import load_dotenv

import uuid
from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams
from langchain_core.runnables import RunnablePassthrough

from src import utils, conf

# Params

In [73]:
conf_settings = conf.load(file="settings.yaml")

LLM_WORKHORSE = conf_settings.llm_workhorse
LLM_FLAGSHIP = conf_settings.llm_flagship
EMBEDDINGS = conf_settings.embeddings
EMB_DIM = conf_settings.embeddings_dim

In [74]:
conf_infra = conf.load(file="infra.yaml")

VDB_URL = conf_infra.vdb_url

# Environment Variables

In [75]:
load_dotenv()

OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
QDRANT_API_KEY = os.environ["QDRANT_API_KEY"]

In [76]:
embeddings = OpenAIEmbeddings(model=EMBEDDINGS)

llm = ChatOpenAI(
    model=LLM_WORKHORSE,
    )


## Ingestion Pipeline

In [77]:
docs = [
    Document(
        page_content="John J. Hopfield and Geoffrey Hinton received the Nobel Prize in Physics in 2024 for their groundbreaking work on artificial neural networks, a foundation of modern AI. Hopfield developed an associative memory model in the 1980s that allows networks to store and reconstruct patterns. Building on this, Hinton developed the Boltzmann machine, which uses statistical physics principles to recognize and classify data. These pioneering contributions are essential for today's machine learning technologies, enhancing applications from medical imaging to material science.",
        metadata={"source": "wikipedia", "topic": "Physics"}
    ),
    Document(
        page_content="In Chemistry, David Baker, Demis Hassabis, and John Jumper were honored win Nobel Prize in 2024 for their breakthroughs in protein structure prediction. Baker’s work in computational protein design enables the creation of novel proteins, while Hassabis and Jumper, known for their work with DeepMind's AlphaFold, developed an AI that accurately predicts protein structures—a long-standing challenge in biology. This advancement could lead to transformative applications in drug development and synthetic biology.",
        metadata={"source": "wikipedia", "topic": "Chemistry"}
    ),
]

client_qdrant = QdrantClient(
    api_key=QDRANT_API_KEY,
    url=VDB_URL

    )

if not client_qdrant.collection_exists("tutorial"):
    client_qdrant.create_collection(
        collection_name="tutorial",
        vectors_config=VectorParams(
            size=EMB_DIM,
            distance=Distance.COSINE),
    )


vector_store = QdrantVectorStore(
    client=client_qdrant,
    collection_name="tutorial",
    embedding=embeddings,
)

uuids = [str(uuid.uuid4()) for _ in range(len(docs))]

vector_store.add_documents(documents=docs, ids=uuids)

['2690cf82-ebfd-48bc-bd52-c61a595a212a',
 '0e8f454e-3ebf-434b-a7cf-26489695bcd0']

In [78]:
docs

[Document(metadata={'source': 'wikipedia', 'topic': 'Physics'}, page_content="John J. Hopfield and Geoffrey Hinton received the Nobel Prize in Physics in 2024 for their groundbreaking work on artificial neural networks, a foundation of modern AI. Hopfield developed an associative memory model in the 1980s that allows networks to store and reconstruct patterns. Building on this, Hinton developed the Boltzmann machine, which uses statistical physics principles to recognize and classify data. These pioneering contributions are essential for today's machine learning technologies, enhancing applications from medical imaging to material science."),
 Document(metadata={'source': 'wikipedia', 'topic': 'Chemistry'}, page_content="In Chemistry, David Baker, Demis Hassabis, and John Jumper were honored win Nobel Prize in 2024 for their breakthroughs in protein structure prediction. Baker’s work in computational protein design enables the creation of novel proteins, while Hassabis and Jumper, know

In [79]:
uuids

['2690cf82-ebfd-48bc-bd52-c61a595a212a',
 '0e8f454e-3ebf-434b-a7cf-26489695bcd0']

In [80]:
vector_store.similarity_search(
    "Nobel Price Physics",
    k=1,
)


[Document(metadata={'source': 'wikipedia', 'topic': 'Physics', '_id': '7392ed9d-1200-42eb-9e3b-f4110dec955f', '_collection_name': 'tutorial'}, page_content="John J. Hopfield and Geoffrey Hinton received the Nobel Prize in Physics in 2024 for their groundbreaking work on artificial neural networks, a foundation of modern AI. Hopfield developed an associative memory model in the 1980s that allows networks to store and reconstruct patterns. Building on this, Hinton developed the Boltzmann machine, which uses statistical physics principles to recognize and classify data. These pioneering contributions are essential for today's machine learning technologies, enhancing applications from medical imaging to material science.")]

In [81]:
vector_store.similarity_search_with_score(
    "Nobel Price Physics",
    k=1,
)


[(Document(metadata={'source': 'wikipedia', 'topic': 'Physics', '_id': '7392ed9d-1200-42eb-9e3b-f4110dec955f', '_collection_name': 'tutorial'}, page_content="John J. Hopfield and Geoffrey Hinton received the Nobel Prize in Physics in 2024 for their groundbreaking work on artificial neural networks, a foundation of modern AI. Hopfield developed an associative memory model in the 1980s that allows networks to store and reconstruct patterns. Building on this, Hinton developed the Boltzmann machine, which uses statistical physics principles to recognize and classify data. These pioneering contributions are essential for today's machine learning technologies, enhancing applications from medical imaging to material science."),
  0.41683024)]

In [82]:
retriever = vector_store.as_retriever(k=1)

retriever.invoke("Nobel Price Physics")

[Document(metadata={'source': 'wikipedia', 'topic': 'Physics', '_id': '7392ed9d-1200-42eb-9e3b-f4110dec955f', '_collection_name': 'tutorial'}, page_content="John J. Hopfield and Geoffrey Hinton received the Nobel Prize in Physics in 2024 for their groundbreaking work on artificial neural networks, a foundation of modern AI. Hopfield developed an associative memory model in the 1980s that allows networks to store and reconstruct patterns. Building on this, Hinton developed the Boltzmann machine, which uses statistical physics principles to recognize and classify data. These pioneering contributions are essential for today's machine learning technologies, enhancing applications from medical imaging to material science."),
 Document(metadata={'source': 'wikipedia', 'topic': 'Physics', '_id': '2690cf82-ebfd-48bc-bd52-c61a595a212a', '_collection_name': 'tutorial'}, page_content="John J. Hopfield and Geoffrey Hinton received the Nobel Prize in Physics in 2024 for their groundbreaking work on

# Query Pipeline

In [83]:
from langchain_core.prompts import ChatPromptTemplate

prompt_template = """Answer the question based only on the following context:
```
{context}
```

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(prompt_template)

prompt.invoke(input=
    {
        "context": ["context1", "context2"],
        "question": "my_question"
    }
).to_messages()

[HumanMessage(content="Answer the question based only on the following context:\n```\n['context1', 'context2']\n```\n\nQuestion: my_question\n", additional_kwargs={}, response_metadata={})]

In [84]:
chain_rag = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
)

chain_rag.invoke("How won the Physics Nobel price in 2024?")

AIMessage(content='John J. Hopfield and Geoffrey Hinton won the 2024 Nobel Prize in Physics for their work on artificial neural networks.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 163, 'prompt_tokens': 632, 'total_tokens': 795, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 128, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-5-mini-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CDfBrn2b994pcLzdiNAgTRPrvKmaD', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--96051084-90df-4165-8b95-de4e9874698b-0', usage_metadata={'input_tokens': 632, 'output_tokens': 163, 'total_tokens': 795, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 128}})

# Improve your RAG

## Parse output

In [85]:
from langchain_core.output_parsers import StrOutputParser

chain_rag = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

chain_rag.invoke("Who won the Physics Nobel price in 2024?")

'John J. Hopfield and Geoffrey Hinton.'

## Format context

In [86]:
from langchain_core.runnables import RunnableLambda


def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])

print((retriever | RunnableLambda(format_docs)).invoke("Who won the Physics Nobel price in 2024?"))

In Chemistry, David Baker, Demis Hassabis, and John Jumper were honored win Nobel Prize in 2024 for their breakthroughs in protein structure prediction. Baker’s work in computational protein design enables the creation of novel proteins, while Hassabis and Jumper, known for their work with DeepMind's AlphaFold, developed an AI that accurately predicts protein structures—a long-standing challenge in biology. This advancement could lead to transformative applications in drug development and synthetic biology.

In Chemistry, David Baker, Demis Hassabis, and John Jumper were honored win Nobel Prize in 2024 for their breakthroughs in protein structure prediction. Baker’s work in computational protein design enables the creation of novel proteins, while Hassabis and Jumper, known for their work with DeepMind's AlphaFold, developed an AI that accurately predicts protein structures—a long-standing challenge in biology. This advancement could lead to transformative applications in drug developm

# Prepare your RAG for evaluation with structured outputs

In [87]:
chain_chat = ( prompt
    | llm
    | StrOutputParser()
)

chain_rag = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | RunnablePassthrough.assign(  # maintains the keys context and question and adds a new one: answer
        answer=chain_chat  # passes context and question as input 
    )
)

chain_rag.invoke("Who won the Chemistry Nobel price in 2024?")

{'context': "In Chemistry, David Baker, Demis Hassabis, and John Jumper were honored win Nobel Prize in 2024 for their breakthroughs in protein structure prediction. Baker’s work in computational protein design enables the creation of novel proteins, while Hassabis and Jumper, known for their work with DeepMind's AlphaFold, developed an AI that accurately predicts protein structures—a long-standing challenge in biology. This advancement could lead to transformative applications in drug development and synthetic biology.\n\nIn Chemistry, David Baker, Demis Hassabis, and John Jumper were honored win Nobel Prize in 2024 for their breakthroughs in protein structure prediction. Baker’s work in computational protein design enables the creation of novel proteins, while Hassabis and Jumper, known for their work with DeepMind's AlphaFold, developed an AI that accurately predicts protein structures—a long-standing challenge in biology. This advancement could lead to transformative applications i

https://python.langchain.com/docs/integrations/vectorstores/qdrant/#hybrid-vector-search
https://python.langchain.com/api_reference/qdrant/qdrant/langchain_qdrant.qdrant.QdrantVectorStore.html